In [1]:
import os, sys
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, './s2spy')
from s2spy import rgdr, RGDR, time
import utils, plotting
path_obs_data = '~/data/volume_2/observational/'
path_subseasonal_data = '/data/volume_2/subseasonal/'

In [4]:
target = xr.open_dataarray(os.path.join(path_obs_data, 'preprocessed', 'chrips_tp_1981-2021_africa_aggregated_intervals.nc'))
target
plotting.plot_maps(target[0], col_dim='target_period')

Plotting Corr maps precip, split 0, target_period 0


TypeError: an integer is required

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f4649f48940> (for post_execute):


TypeError: an integer is required

TypeError: an integer is required

<Figure size 2378.48x250 with 10 Axes>

In [3]:
hoa_mask = xr.open_dataarray(os.path.join(path_obs_data, 'chirps_hoa_dry_mask_0.25deg.nc'))
target_timeseries = target.where(hoa_mask).mean(dim=('latitude', 'longitude'))

In [4]:
dfs = pd.read_hdf('~/data/volume_2/subseasonal/ecmwf/aggregated/aggregation_timestamps.h5').sort_index()
df = dfs[['aggregation_start_inclusive', 'aggregation_end_inclusive']].copy()
sel_months = [10, 11, 12]
mon_start_mask = [True if m in sel_months else False for m in df.set_index('aggregation_start_inclusive').index.month]
mon_end_mask = [True if m in sel_months else False for m in df.set_index('aggregation_end_inclusive').index.month]
mon_mask = np.logical_and(mon_start_mask, mon_end_mask)
df_sub = dfs[mon_mask][dfs[mon_mask].index.year==dfs[mon_mask].index.year[0]]
anchor_dates = [(d.month, d.day) for d in df_sub.index]


In [5]:
anchor_date = anchor_dates[0]

In [8]:
cal = time.AdventCalendar(anchor=anchor_date, freq='30d')
cal = cal.map_years(1981, 2021)

In [48]:
sst = xr.open_dataarray(os.path.join(path_obs_data, 'preprocessed', 'era5_sst_1959-2021_1_12_daily_2.0deg.nc'))
sst['time'] = ('time', pd.to_datetime(sst.time.values))
sst_resampled = time.resample(cal, sst)
sst_resampled

<xarray.Dataset>
Dimensions:      (anchor_year: 41, i_interval: 12, longitude: 180, latitude: 91)
Coordinates:
    index        (anchor_year, i_interval) int64 0 1 2 3 4 ... 488 489 490 491
    interval     (anchor_year, i_interval) object (1981-08-18, 1981-09-17] .....
  * longitude    (longitude) float32 -180.0 -178.0 -176.0 ... 174.0 176.0 178.0
  * latitude     (latitude) float32 -90.0 -88.0 -86.0 -84.0 ... 86.0 88.0 90.0
    mask         (latitude, longitude) bool ...
  * anchor_year  (anchor_year) int64 1981 1982 1983 1984 ... 2018 2019 2020 2021
  * i_interval   (i_interval) int64 0 1 2 3 4 5 6 7 8 9 10 11
    target       (i_interval) bool True False False False ... False False False
Data variables:
    sst          (anchor_year, i_interval, latitude, longitude) float64 nan ....

In [15]:
for period in target_timeseries.target_period.values[::4]:
    corr, pval = rgdr.rgdr.correlation(sst_resampled['sst'].sel(i_interval=3), target_timeseries.sel(target_period=int(period)),
                                       corr_dim='anchor_year')
    plotting.plot_maps(corr.where(pval < 0.05))

Plotting Corr maps None, split 0, lag 0


TypeError: an integer is required

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fbd1479ddc0> (for post_execute):


TypeError: an integer is required

TypeError: an integer is required

<Figure size 594.505x250 with 1 Axes>

In [14]:
for period in target_timeseries.target_period.values[::4]:
    rg = RGDR(target_timeseries.sel(target_period=0), eps_km=600, alpha=0.05, min_area_km2=3000**2)
    rg.plot_clusters(sst_resampled['sst'].sel(i_interval=1))

TypeError: __init__() got multiple values for argument 'eps_km'